# Clasificación multinomial

### Modelamiento por medio de redes neuronales

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.1.0


In [6]:
plt.rcParams['figure.figsize']=(12,0)

---
### ¿Cuál es el objetivo?

Realizar una máquina que sea capaz de clasififcar a las estrellas, glaxias y quasar por medio de una red neuronal, de modo que cuando se le presente una observación especificque que tipo de elemento del espacio es.

---

In [7]:
url = 'https://raw.githubusercontent.com/joaramirezcu/Mineria_Proyecto/master/Skyserver_SQL2_27_2018%206_51_39%20PM.csv'
data_f1 = pd.read_csv(url, error_bad_lines=False)
data_f1=data_f1.drop(['rerun'], axis=1)
data_f1.head()

,objid,ra,dec,u,g,r,i,z,run,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,4,269,3.722370e+18,STAR,0.000590,3306,54922,512


Veamos si tiene valores nulos y ceros

In [8]:
data_f1.isnull().sum()

objid        0
ra           0
dec          0
u            0
g            0
r            0
i            0
z            0
run          0
camcol       0
field        0
specobjid    0
class        0
redshift     0
plate        0
mjd          0
fiberid      0
dtype: int64

In [9]:
(data_f1 != 0).apply(pd.Series.value_counts)

,objid,ra,dec,u,g,r,i,z,run,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN
True,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,9981,10000.0,10000.0,10000.0


In [10]:
#CONTANDO LOS VALORES TRUE DE UNA SERIE

s=(data_f1['redshift'] != 0)
print(s)
boolarr = np.array(s, dtype=np.bool)
boolarr
np.sum(boolarr)

0       True
1       True
2       True
3       True
4       True
        ... 
9995    True
9996    True
9997    True
9998    True
9999    True
Name: redshift, Length: 10000, dtype: bool


9981

## Separando los features de las target

In [11]:
X_data = data_f1[[m for m in data_f1.columns if m not in ['class']]]
Y_data = data_f1[['class']]                                                        #  ]] es Para volverlo DataFrame

In [12]:
print(X_data.shape, Y_data.shape)

(10000, 16) (10000, 1)


## Dividiendo los datos de entrenamiento y de validación

In [13]:
X_train , X_test ,Y_train, Y_test = train_test_split(X_data , Y_data , test_size=0.2)

In [32]:
print('La dimensión de los las features de entrenamiento es: ', X_train.shape, '  y es de tipo:', type(X_train))
print('La dimensión del label de entrenamiento es: ', Y_train.shape, '  y es de tipo:', type(Y_train))
print('La dimensión de los las features de test es:  ', X_test.shape, '  y es de tipo:', type(X_test))
print('La dimensión del label de test es:  ', Y_test.shape, '  y es de tipo:', type(Y_test))

La dimensión de los las features de entrenamiento es:  (8000, 16)   y es de tipo: <class 'pandas.core.frame.DataFrame'>
La dimensión del label de entrenamiento es:  (8000, 1)   y es de tipo: <class 'pandas.core.frame.DataFrame'>
La dimensión de los las features de test es:   (2000, 16)   y es de tipo: <class 'pandas.core.frame.DataFrame'>
La dimensión del label de test es:   (2000, 1)   y es de tipo: <class 'pandas.core.frame.DataFrame'>


## Diseño de la Red Neuronal (para un modelo Logístico Multinomial)

Como el objetivo es clasificar las imágenes astronomicas detectadas por el telescopio SDSS transformadas en 16 atributos, se procede a establecer una diseño de la red neuronal de la siguiente manera:

* Cada observación del telescopio se presentará a la entrada de la red neuronal como una tensor de una dimensión (1D) y de tamaño $16$. Es por esto que el número de neuronas en la primera capa estará determinada por 16 y la capa de salida tendrá 3 neuronas esto es porque se quiere clasificar en **_STAR, QUASAR, QSO_**.

* En esta red neuronal se considerará el caso en que cada una de las 16 neuronas de entrada se consectará con cada una de las 3 neuronas de salida. 

Así, la matriz de pesos $\mathbf{W}$ tendrá tamaño $784 \times 10$ y el vector $\mathbf{b}$ de interceptos (bias) será de un vector tamaño 10.


En nuestro primer modelo, la función de activación de la capa de entrada será la identidad y en la capa de salida la función de activación será la función *softmax* que definimos abajo.


## Construyendo pipeline 

In [36]:
def feed_input(features_dataframe, target_dataframe, num_of_epochs=10, shuffle=True, batch_size=32):
  def input_feed_function():
    dataset = tf.data.Dataset.from_tensor_slices((dict(features_dataframe), target_dataframe))
    if shuffle:
      dataset = dataset.shuffle(2000)
    dataset = dataset.batch(batch_size).repeat(num_of_epochs)
    return dataset
  return input_feed_function

train_feed_input = feed_input(X_train, Y_train)
train_feed_input_testing = feed_input(X_train, Y_train, num_of_epochs=1, shuffle=False)
test_feed_input = feed_input(X_test, Y_test, num_of_epochs=1, shuffle=False)

La anterior tuberia se realiaz con el fin de alistar las observaciones de entrenamiento y test para que sean tomadas por *tensorflow*

In [38]:
type(train_feed_input_testing)

function

In [42]:
feature_columns_numeric = [tf.feature_column.numeric_column(m) for m in X_train.columns]
print(type(feature_columns_numeric))

<class 'list'>


## Inicialización de los pesos

In [22]:
import math

weights = tf.Variable(tf.random.normal((16,3))/math.sqrt(16), dtype= tf.float32)
bias = tf.Variable(tf.zeros(3,tf.float32))

In [28]:
X_train = tf.constant(X_train, dtype= tf.float32)
X_test = tf.constant(x_test, dtype= tf.float32)


print("shape x_train = ",X_train.shape)
print("type x_train = ",X_train.dtype)
print("Min x_train = ",tf.reduce_min(X_train).numpy())
print("Max x_train = ",tf.reduce_max(X_train).numpy())

# transform y to int32 and create constant tensors
y_train = tf.constant(Y_train,dtype=tf.int32)
y_test = tf.constant(Y_test,dtype=tf.int32)

print("type y_train = ",Y_train.dtype)
print("Min y_train = ",tf.reduce_min(Y_train).numpy())
print("Max y_train = ",tf.reduce_max(Y_train).numpy())

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [23]:
def log_softmax(x):
    eta = tf.math.reduce_max(x)
    return x - eta - tf.math.log(tf.math.reduce_sum(tf.math.exp(x-eta), -1, keepdims=True))

def model(xb):
    return log_softmax(xb @ weights + bias) # z = x'W +b

In [24]:
bs = 64 # batch size  (lote)

xb = X_train[0:bs]
preds = model(xb) # predictions
print('Predicciones para la primera imagen: ',preds[0],'\n Tamaño de la matriz de predicciones de este lote: ', preds.shape)

IndexError: tuple index out of range

In [161]:
{'uno' : pd.Series([1, 2, 3], index=['a', 'b', 'c'])}

{'uno': a    1
 b    2
 c    3
 dtype: int64}

In [165]:
dataframe = pd.DataFrame({'uno' : pd.Series([1, 2, 3], index=['a', 'b', 'c']),
                           'dos' : pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])   })
print(dataframe)

   uno  dos
a  1.0    1
b  2.0    2
c  3.0    3
d  NaN    4


In [166]:
dataframe.loc['a']

uno    1.0
dos    1.0
Name: a, dtype: float64

In [167]:
dataframe.iloc[1]

uno    2.0
dos    2.0
Name: b, dtype: float64

In [172]:
serie = pd.Series([1,2,3,4], index=['a','b','c','d'])
print(serie)
print(serie[[True, False, True, False]])

a    1
b    2
c    3
d    4
dtype: int64
a    1
c    3
dtype: int64


* https://ligdigonzalez.com/manipulando-datos-perdidos-en-python/
* https://ligdigonzalez.com/cambiando-variables-categorias-por-variables-numericas-con-python/